In [13]:
import os
import nibabel as nib
import numpy as np
import plotly.graph_objects as go

val_path = "../data/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData"
flair_files = [f for f in os.listdir(val_path) if f.endswith("_flair.nii")]

if not flair_files:
    raise FileNotFoundError("No _flair.nii files found in the validation directory.")

flair_file = flair_files[0]
sample_id = flair_file.split("_flair.nii")[0]
flair_path = os.path.join(val_path, flair_file)

try:
    img = nib.load(flair_path)
    img_data = img.get_fdata()
except Exception as e:
    raise RuntimeError(f"Failed to load or read NIfTI file: {flair_path}\n{e}")

fig = go.Figure(data=go.Volume(
    x=np.linspace(0, img_data.shape[0] - 1, img_data.shape[0]),
    y=np.linspace(0, img_data.shape[1] - 1, img_data.shape[1]),
    z=np.linspace(0, img_data.shape[2] - 1, img_data.shape[2]),
    value=img_data.flatten(),
    opacity=0.1,
    surface_count=15,
))
fig.update_layout(title=f"{sample_id} - FLAIR MRI", width=800, height=800)
fig.show()

FileNotFoundError: No _flair.nii files found in the validation directory.